<a href="https://colab.research.google.com/github/rohailkhan/Deep-Learning-with-Python-Jb/blob/master/4_Deep_learning_Sonar_data_classification_using_Pipeline_list_of_estimators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# MLP with automatic validation set
from keras.models import Sequential
from keras.optimizers import adam_v2
from keras.layers import Dense,Conv2D, BatchNormalization,MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, Flatten
import numpy as np
import pandas as pd
# fix random seed for reproducibility
seed = 7

np.random.seed(seed)


In [2]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [3]:
path='https://raw.githubusercontent.com/rohailkhan/data/main/sonar.csv'
sonar=pd.read_csv(path)

In [4]:
sonar.head()

,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,0.1609,0.1582,0.2238,0.0645,0.0660,0.2273,0.3100,0.2999,0.5078,0.4797,0.5783,0.5071,0.4328,0.5550,0.6711,0.6415,0.7104,0.8080,0.6791,0.3857,0.1307,0.2604,0.5121,0.7547,0.8537,0.8507,0.6692,0.6097,0.4943,0.2744,0.0510,0.2834,0.2825,0.4256,0.2641,0.1386,0.1051,0.1343,0.0383,0.0324,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
0,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,0.4918,0.6552,0.6919,0.7797,0.7464,0.9444,1.0000,0.8874,0.8024,0.7818,0.5212,0.4052,0.3957,0.3914,0.3250,0.3200,0.3271,0.2767,0.4423,0.2028,0.3788,0.2947,0.1984,0.2341,0.1306,0.4182,0.3835,0.1057,0.1840,0.1970,0.1674,0.0583,0.1401,0.1628,0.0621,0.0203,0.0530,0.0742,0.0409,0.0061,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
1,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,0.6333,0.7060,0.5544,0.5320,0.6479,0.6931,0.6759,0.7551,0.8929,0.8619,0.7974,0.6737,0.4293,0.3648,0.5331,0.2413,0.5070,0.8533,0.6036,0.8514,0.8512,0.5045,0.1862,0.2709,0.4232,0.3043,0.6116,0.6756,0.5375,0.4719,0.4647,0.2587,0.2129,0.2222,0.2111,0.0176,0.1348,0.0744,0.0130,0.0106,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
2,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,0.0881,0.1992,0.0184,0.2261,0.1729,0.2131,0.0693,0.2281,0.4060,0.3973,0.2741,0.3690,0.5556,0.4846,0.3140,0.5334,0.5256,0.2520,0.2090,0.3559,0.6260,0.7340,0.6120,0.3497,0.3953,0.3012,0.5408,0.8814,0.9857,0.9167,0.6121,0.5006,0.3210,0.3202,0.4295,0.3654,0.2655,0.1576,0.0681,0.0294,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
3,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,0.4152,0.3952,0.4256,0.4135,0.4528,0.5326,0.7306,0.6193,0.2032,0.4636,0.4148,0.4292,0.5730,0.5399,0.3161,0.2285,0.6995,1.0000,0.7262,0.4724,0.5103,0.5459,0.2881,0.0981,0.1951,0.4181,0.4604,0.3217,0.2828,0.2430,0.1979,0.2444,0.1847,0.0841,0.0692,0.0528,0.0357,0.0085,0.0230,0.0046,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
4,0.0286,0.0453,0.0277,0.0174,0.0384,0.0990,0.1201,0.1833,0.2105,0.3039,0.2988,0.4250,0.6343,0.8198,1.0000,0.9988,0.9508,0.9025,0.7234,0.5122,0.2074,0.3985,0.5890,0.2872,0.2043,0.5782,0.5389,0.3750,0.3411,0.5067,0.5580,0.4778,0.3299,0.2198,0.1407,0.2856,0.3807,0.4158,0.4054,0.3296,0.2707,0.2650,0.0723,0.1238,0.1192,0.1089,0.0623,0.0494,0.0264,0.0081,0.0104,0.0045,0.0014,0.0038,0.0013,0.0089,0.0057,0.0027,0.0051,0.0062,R


In [5]:
sonar=pd.read_csv(path,header=None)
sonar.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,0.1609,0.1582,0.2238,0.0645,0.0660,0.2273,0.3100,0.2999,0.5078,0.4797,0.5783,0.5071,0.4328,0.5550,0.6711,0.6415,0.7104,0.8080,0.6791,0.3857,0.1307,0.2604,0.5121,0.7547,0.8537,0.8507,0.6692,0.6097,0.4943,0.2744,0.0510,0.2834,0.2825,0.4256,0.2641,0.1386,0.1051,0.1343,0.0383,0.0324,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,0.4918,0.6552,0.6919,0.7797,0.7464,0.9444,1.0000,0.8874,0.8024,0.7818,0.5212,0.4052,0.3957,0.3914,0.3250,0.3200,0.3271,0.2767,0.4423,0.2028,0.3788,0.2947,0.1984,0.2341,0.1306,0.4182,0.3835,0.1057,0.1840,0.1970,0.1674,0.0583,0.1401,0.1628,0.0621,0.0203,0.0530,0.0742,0.0409,0.0061,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,0.6333,0.7060,0.5544,0.5320,0.6479,0.6931,0.6759,0.7551,0.8929,0.8619,0.7974,0.6737,0.4293,0.3648,0.5331,0.2413,0.5070,0.8533,0.6036,0.8514,0.8512,0.5045,0.1862,0.2709,0.4232,0.3043,0.6116,0.6756,0.5375,0.4719,0.4647,0.2587,0.2129,0.2222,0.2111,0.0176,0.1348,0.0744,0.0130,0.0106,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,0.0881,0.1992,0.0184,0.2261,0.1729,0.2131,0.0693,0.2281,0.4060,0.3973,0.2741,0.3690,0.5556,0.4846,0.3140,0.5334,0.5256,0.2520,0.2090,0.3559,0.6260,0.7340,0.6120,0.3497,0.3953,0.3012,0.5408,0.8814,0.9857,0.9167,0.6121,0.5006,0.3210,0.3202,0.4295,0.3654,0.2655,0.1576,0.0681,0.0294,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,0.4152,0.3952,0.4256,0.4135,0.4528,0.5326,0.7306,0.6193,0.2032,0.4636,0.4148,0.4292,0.5730,0.5399,0.3161,0.2285,0.6995,1.0000,0.7262,0.4724,0.5103,0.5459,0.2881,0.0981,0.1951,0.4181,0.4604,0.3217,0.2828,0.2430,0.1979,0.2444,0.1847,0.0841,0.0692,0.0528,0.0357,0.0085,0.0230,0.0046,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


## Label encoding of the target
This will be done in 2 steps.
First the string classes are converted to integers by LabelEncoder


In [6]:
dataset = sonar.values
# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

In [7]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)

LabelEncoder()

In [8]:
encoded_Y = encoder.transform(Y)

# Base line Model

In [9]:
from keras.layers import BatchNormalization
from keras.layers.core import Dense, Dropout, Activation 

In [10]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=60,kernel_initializer='normal',activation='relu'))
    model.add(Dense(1,kernel_initializer='normal',  activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Make a wrapper to for Keras Model for use in Scikit-learn

In [11]:
estimator = KerasClassifier(build_fn=baseline_model, nb_epoch=100, batch_size=5, verbose=0)

# Evaluate model using k-fold cross validation
## StratifiedKFold is not meant to be used for multilabel targets . It needs a 1D-array to determine how to split the indices.

In [12]:
#  from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Results

In [13]:
results1 = cross_val_score(estimator, X, encoded_Y , cv=kfold)


In [14]:
print("Accuracy: %.2f%% (%.2f%%)" % (results1.mean()*100, results1.std()*100))

Accuracy: 54.81% (3.01%)


In [15]:
# all model written together
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=60,kernel_initializer='normal',activation='relu'))
    model.add(Dense(1,kernel_initializer='normal',  activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
    
estimator = KerasClassifier(build_fn=baseline_model, nb_epoch=100, batch_size=5, verbose=0)

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

results1 = cross_val_score(estimator, X, encoded_Y , cv=kfold)

# Improve Performance With Data Preparation
## Adding another step of Standard scalor **(within the pass of a cross validation run and to use the trained standardization instance to prepare the unseen test fold. This makes standardization a step in model preparation in the cross validation process and it prevents the algorithm having knowledge of unseen data during evaluation)** This is acheived by Pipeline

An eective data
preparation scheme for tabular data when building neural network models is standardization.
This is where the data is rescaled such that the mean value for each attribute is 0 and the standard
deviation is 1. This preserves Gaussian and Gaussian-like distributions whilst normalizing the
central tendencies for each attribute.
We can use scikit-learn to perform the standardization of our Sonar dataset using the
StandardScaler class. Rather than performing the standardization on the entire dataset, it is
good practice to train the standardization procedure on the training data within the pass of a cross validation run and to use the trained standardization instance to prepare the unseen test
fold. This makes standardization a step in model preparation in the cross validation process
and it prevents the algorithm having knowledge of unseen data during evaluation, knowledge
that might be passed from the data preparation scheme like a crisper distribution.
We can achieve this in scikit-learn using a Pipeline class. The pipeline is a wrapper that
executes one or more models within a pass of the cross validation procedure. Here, we can
dene a pipeline with the StandardScaler followed by our neural network model.

# Using standard scalor and Kfold validation together with Pipeline list of estimaters

In [16]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.model_selection import cross_val_score

In [18]:
def create_baseline():
# create model
  model = Sequential()
  model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
# evaluate baseline model with standardized dataset

estimators = []
estimators.append(('standardiz', StandardScaler()))
estimators.append(('ml', KerasClassifier(build_fn=create_baseline, nb_epoch=100,
batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 73.02% (11.29%)


In [27]:
def create_baseline():
# create model
  model = Sequential()
  
  model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu'))
  model.add(Dropout(0.2))
  
  model.add(Dense(120, kernel_initializer='normal', activation='relu'))
  model.add(Dropout(0.2))
  

  model.add(Dense(120, kernel_initializer='normal', activation='relu'))
  

  

  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
# evaluate baseline model with standardized dataset

estimators = []
estimators.append(('standardiz', StandardScaler()))
estimators.append(('ml', KerasClassifier(build_fn=create_baseline, nb_epoch=100,
batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 78.38% (7.53%)


In [31]:
from tensorflow.keras.optimizers import RMSprop ,SGD ,Adam

## Adding more layers and also dropout layers

In [36]:
def create_baseline():
# create model
  model = Sequential()
  
  model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu'))
  model.add(Dropout(0.2))
  
  model.add(Dense(120, kernel_initializer='normal', activation='relu'))
  model.add(Dropout(0.2))
  

  model.add(Dense(120, kernel_initializer='normal', activation='relu'))
  
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

  adam=Adam(learning_rate=0.01)
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
  return model
# evaluate baseline model with standardized dataset

estimators = []
estimators.append(('standardiz', StandardScaler()))
estimators.append(('ml', KerasClassifier(build_fn=create_baseline, nb_epoch=100,
batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 80.76% (8.64%)


# Tips For Using Dropout
The original paper on Dropout provides experimental results on a suite of standard machine
learning problems. As a result they provide a number of useful heuristics to consider when using
dropout in practice:


1.   List itemGenerally use a small dropout value of 20%-50% of neurons with 20% providing a 
good starting point. A probability too low has minimal eect and a value too high results in under-learning by the network.

1.   List itemUse a larger network. You are likely to get better performance when dropout is used
on a larger network, giving the model more of an opportunity to learn independent
representations.

1.   List itemUse dropout on input (visible) as well as hidden layers. Application of dropout at each
layer of the network has shown good results.

1.   List itemUse a large learning rate with decay and a large momentum. Increase your learning rate
by a factor of 10 to 100 and use a high momentum value of 0.9 or 0.99.

1.   List itemConstrain the size of network weights. A large learning rate can result in very large
network weights. Imposing a constraint on the size of network weights such as max-norm
regularization with a size of 4 or 5 has been shown to improve results.

### Question : When evaluating, is the split in train and test data necesarry? No, crossvalidation in itself makes that obsolete. 
# **BUT**
## In case you want to do try different models and then select one best model and want to do predictions with it, then you need a Test part

In [ ]:
results.view() , results.shape 

(array([0.71428573, 0.76190478, 0.80952382, 0.76190478, 0.61904764,
        0.90476191, 0.80952382, 0.66666669, 0.89999998, 0.44999999]),
 (10,),
 <function ndarray.trace>)

# Result : The performance has increased using 

# How the Pipeline works?
## Pipeline of transforms with a final estimator.

Pipeline  sequentially applies a list of transforms and a final estimator.

**Intermediate steps in form a list [   ] .** 

Intermediate steps (in form a list) of the pipeline must be 'transforms' *(e.g StandardScalor() KerasClassifier etc)*, that is, they must implement fit and transform methods.

**The final estimator**

only needs to implement fit.

The purpose of the pipeline is to assemble several steps that can be
cross-validated together while setting different parameters.

For this, it enables setting parameters of the various steps using their
names and the parameter name separated by a '__', as in the example below.

A step's estimator may be replaced entirely by setting the parameter
with its name to another estimator, or a transformer removed by setting
it to 'passthrough' or None
### Note : We cannot add the kfold validation in the Pipeline because All intermediate steps should be transformers and implement fit 